In [1]:
import sys
import os
from pathlib import Path
#
path_root = Path(os.path.abspath(''))
sys.path.insert(1, os.path.join(path_root))
#
path_root2 = Path(os.path.abspath('')).parent
sys.path.insert(1, os.path.join(path_root2))

import time
import talib
import sqlite3 as sqlite3
import pandas as pd
from util.util import get_ucodes

path_sqlite = os.path.join(path_root, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
if not os.path.exists(path_sqlite):
    path_sqlite = os.path.join(path_root2, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')

In [2]:
map = {
    '上證50': '000016.SS',
    '新华富时A50指数': 'xin9x000'
}

for k, v in map.items():
    df = pd.read_csv(f'{k}.csv')
    df['Date']= pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
    df = df.sort_values(by='Date', ascending=True)
    df.columns = ['stime', 'open', 'high', 'low', 'close']
    df['code'] = v
    df['volume'] = 0

    conn = sqlite3.connect(path_sqlite)
    tb_name = 's_' + v.lower().replace('.', '').replace('ss', '').replace('sz', '')

    conn.execute("DROP TABLE %s" % tb_name)
    conn.execute("CREATE TABLE IF NOT EXISTS %s (stime TEXT PRIMARY KEY, code TEXT NOT NULL, close REAL, "
                 "high REAL, low REAL, open REAL, volume REAL)" % tb_name)
    
    stmt = f"INSERT OR REPLACE INTO {tb_name} (stime, open, high, low, close, code, volume) VALUES (?, ?, ?, ?, ?, ?, ?)"
    conn.executemany(stmt, df.values)
    
    conn.commit()
    conn.close()